Pyspark consumer for streaming data

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import time

kafka_topic_name = "songTopic"
kafka_bootstrap_servers = 'localhost:9092'

#spark version = 3.5.3
#scala version = 2.12.18

spark = SparkSession \
        .builder \
        .appName("Spotify Streaming Reccomendation System") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4") \
        .getOrCreate()\


spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/home/lamp/Projects/big-data/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lamp/.ivy2/cache
The jars for the packages stored in: /home/lamp/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6b7d74bd-9eb3-4a71-bd9d-c14d02b0143a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 205ms :: artifacts dl 6ms
	:: modu

In [2]:
# Construct a streaming DataFrame that reads from test-topic
songs_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic_name) \
    .option("startingOffsets", "latest") \
    .load()

In [3]:
songs_df1 = songs_df.selectExpr("CAST(value AS STRING)", "timestamp")


songs_schema_string = "order_id INT,id STRING, name STRING,popularity INT, duration_ms DOUBLE, explicit INT, " \
                        + "artists STRING, id_artists STRING, release_date STRING, " \
                        + "danceability DOUBLE," \
                        + "energy DOUBLE, key INT, loudness DOUBLE, " \
                        + "mode INT," \
                        + "speechiness DOUBLE," \
                        + "acousticness DOUBLE, instrumentalness DOUBLE, liveness DOUBLE, " \
                        + "valence DOUBLE, tempo DOUBLE, time_signature DOUBLE"



songs_df2 = songs_df1 \
        .select(from_csv(col("value"), songs_schema_string) \
                .alias("song"), "timestamp")


songs_df3 = songs_df2.select("song.*", "timestamp")

Spark SQL View

In [4]:
songs_df3.createOrReplaceTempView("song_find")
song_find_text = spark.sql("SELECT * FROM song_find")
songs_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("testedTable5") \
        .start()

songs_agg_write_stream.awaitTermination(1)

False

In [5]:
# pip install spotipy --upgrade

Favorite song data generated using Spotify API

In [6]:
from spotify_api import getSong

song_data = getSong.passs()
#song_data.rename(columns={'duration_s': 'duration_ms' }, inplace=True)
song_data = song_data.drop(['id', 'added_at', 'time_signature','duration_s'], axis='columns')
rand_n = random.randint(0, len(song_data) - 1)
add_df = song_data.head(rand_n)[-1:]

Gtk-Message: 20:36:41.903: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1N1ZpYUJc9fwrqk53FGgWv with Params: {} returned 403 due to None


4 tracks


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1N1ZpYUJc9fwrqk53FGgWv:
 None, reason: None

Feature Engineering

In [ ]:
df = spark.sql("SELECT * FROM testedTable5")

df = df.sort(df.release_date.desc())

df_stream = df

df = df.drop('order_id',
'id',
'explicit',
'mode',
'release_date',
'id_artists',
'time_signature',
'duration_ms',
'timestamp')

df_sp = spark.createDataFrame(add_df)
df = df.union(df_sp)

from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=[
'danceability',
'energy',
'loudness',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo'], outputCol='features')
assembled_data=assembler.setHandleInvalid("skip").transform(df)

NameError: name 'add_df' is not defined